In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/1.part_number'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

from urllib.parse import urlparse, parse_qs

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

list_series_error, df_input = [], pd.read_excel('./file/input.xlsx',
                                                header=0,
                                                dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_series_error, count_remain

    (crawler_index, crawler_series) = crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        url_parse = urlparse(crawler_series['Url'])

        dict_param = parse_qs(url_parse.query)

        list_dict_correct, page = [], 0
        while True:

            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request = f'''https://ecatalog.smpcorp.com/v2/fs/api/part/partsearch?filter={dict_param['part'][0]}&filterType={dict_param['search'][0] if 'search' in dict_param else 'n'}&searchType={dict_param['type'][0] if 'type' in dict_param else 'null'}&imageSize=80&start={page * 96}&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-'''

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        list_dict = resp.json()

                        if list_dict:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            if page == 0:
                count = list_dict[0]['maxRows']

            # = = = = = = = = = = = = = = =

            list_dict_correct.extend([{'No': dict_['rowId'],
                                       'Part Number': dict_['basePart'],
                                       'Url': f'''https://www.4s.com/en/ecatalog?part={dict_['webBase'].strip()}&type=p&search=s'''} for dict_ in list_dict])

            # = = = = = = = = = = = = = = =

            if count_remain % 30 == 0 and count_remain != 0:
                clear_output()
            print(f'''[第{page + 1}页] - [请求{count_retry}次] - [{len(list_dict_correct)}/{count}] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

            # = = = = = = = = = = = = = = =

            page += 1

            # = = = = = = = = = = = = = = =

            if len(list_dict_correct) == count:
                break

        # = = = = = = = = = = = = = = =

        df_correct = pd.DataFrame(list_dict_correct)
        df_correct.to_excel(f'''{directory_out}/crawler_{crawler_series['No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./file/part_number_error.xlsx', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[第1页] - [请求7次] - [96/719] - [剩余1条] - [2024/10/29 14:21:11] - 2. https://ecatalog.smpcorp.com/V2/fs/api/part/partsearch?filter=Heater Core&filterType=s&searchType=p&imageSize=80&start=0&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-

[第1页] - [请求2次] - [96/727] - [剩余1条] - [2024/10/29 14:21:12] - 1. https://ecatalog.smpcorp.com/V2/fs/api/part/partsearch?filter=Evaporator Core&filterType=s&searchType=p&imageSize=80&start=0&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-

[第2页] - [请求7次] - [192/719] - [剩余1条] - [2024/10/29 14:21:14] - 2. https://ecatalog.smpcorp.com/V2/fs/api/part/partsearch?filter=Heater Core&filterType=s&searchType=p&im

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3033.85it/s]

Done ~
